## Preparation

### Installation

In [ ]:
# Building sources
!git clone git@github.com:oaqa/FlexNeuART.git

In [ ]:
cd FlexNeuART

In [ ]:
!./install_packages.sh

In [ ]:
!./build.sh

### Downloading demo data

In [ ]:
!wget boytsov.info/datasets/flecsneurt-demo-2020-04-07.tar.bz2 

In [ ]:
# Unpacking it
!tar jxvf flecsneurt-demo-2020-04-07.tar.bz2

In [ ]:
# Creating a Lucene index
!scripts/index/create_lucene_index.sh squad

In [ ]:
#!creating a forward index for two fields:
# text is a parsed text field
# text_raw is a raw text field that keeps the text as is
# -clean removes all previous forward indices
!scripts/index/create_fwd_index.sh squad mapdb  \
                               "text:parsedText text_unlemm:raw" \
                               -clean

## API demo

In [ ]:
from scripts.py_flexneuart.setup import *

In [ ]:
# add Java JAR to the class path
configure_classpath('target')

In [ ]:
# create a resource manager
resource_manager=create_featextr_resource_manager('collections/squad/forward_index')

### Retrieval

In [ ]:
from scripts.pyflexneuart.cand_provider import *
# create a candidate provider/generator
cand_prov = create_cand_provider(resource_manager, PROVIDER_TYPE_LUCENE, 'collections/squad/lucene_index')

In [ ]:
run_query(cand_prov, 20, "university notre dame student run".split())

#### Forward index demo

In [ ]:
from scripts.pyflexneuart.fwd_index import get_forward_index

### First let's play with a raw index that keeps ony unparsed text

In [ ]:
raw_indx = get_forward_index(resource_manager, 'text_unlemm')

In [ ]:
# the raw flag is set
raw_indx.is_raw

In [ ]:
raw_indx.get_doc_raw('@4302')

### A parsed index has more info

In [ ]:
parsed_indx = get_forward_index(resource_manager, 'text')

In [ ]:
# here is_raw is False
parsed_indx.is_raw

In [ ]:
parsed_indx.get_doc_parsed('@4302')

In [ ]:
# Let's extract the first document word and its info
parsed_indx.get_word_by_id(10063), parsed_indx.get_word_entry_by_id(10063)